In [13]:
%matplotlib inline

import numpy as np
import datetime as dt
import pandas as pd
import os
import xarray as xr
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, save
from bokeh.io import reset_output
import bokeh
import properscoring as ps

In [25]:
# ===================================================== #
# Parameters
# ===================================================== #
# --- USGS data --- #
# Site info file; needs to have columns "short_name" and "site_no"
site_info_csv = '/civil/hydro/ymao/data_assim/data/USGS/Crow_2017/sites_in_ArkRed/site_info_in_ArkRed.csv'
# Directory of USGS data
usgs_data_dir = '/civil/hydro/ymao/data_assim/data/USGS/Crow_2017/sites_in_ArkRed/'

# --- Synthetic data --- #
synth_openloop_data_dir = \
    '/civil/hydro/ymao/data_assim/tools/plot_analyze_results/' \
    'output/20180503.ArkRed.synthetic.LAI_from_veglib/' \
    'sm_0.5.prec_0.3.N32.no_sm3_update_only/synthetic/data'
synth_data_dir = '/civil/hydro/ymao/data_assim/tools/plot_analyze_results/' \
                 'output/20180503.ArkRed.synthetic.LAI_from_veglib/' \
                 'sm_0.5.prec_0.3.N32.no_sm3_update_only/EnKF/data'
synth_zero_updata_data_dir = \
    '/civil/hydro/ymao/data_assim/tools/plot_analyze_results/' \
    'output/20180503.ArkRed.synthetic.LAI_from_veglib/' \
    'sm_0.5.prec_0.3.N32.no_sm3_update_only/EnKF.zero_update/data'

# --- Routed --- #
# Openloop nc
openloop_nc = '/civil/hydro/ymao/data_assim/output/RVIC/ArkRed.AndyWood_rout/' \
              'openloop.NLDAS2.2015_2017.LAI_from_veglib/hist/' \
              'openloop.rvic.h0a.2018-01-01-00.nc'
# Ensemble size
N = 32
# Ensemble nc; "{}" will be replaced by ensemble index
ensemble_basenc = '/civil/hydro/ymao/data_assim/output/RVIC/ArkRed.AndyWood_rout/' \
                  'ArkRed.smap.NLDAS2.qc_no_winter.LAI_from_veglib/' \
                  'sm_0.5.prec_0.3.N32.no_sm3_update_only/EnKF/' \
                  'hist/ens{}.rvic.h0a.2018-01-01-00.nc'

# Time lag of routed data with local time [hours];
# Example: if local time is UTC-6 (ArkRed) and routed data is in UTC, then time_lag = 6
time_lag = 6

# --- Zero-update --- #
# Zero-update ensemble nc; "{}" will be replaced by ensemble index
zero_update_ensemble_basenc = \
    '/civil/hydro/ymao/data_assim/output/RVIC/ArkRed.AndyWood_rout/' \
    'ArkRed.smap.NLDAS2.qc_no_winter.LAI_from_veglib/' \
    'sm_0.5.prec_0.3.N32.no_sm3_update_only/EnKF.zero_update/' \
    'hist/ens{}.rvic.h0a.2018-01-01-00.nc'

# --- Time --- #
start_time = pd.to_datetime('2015-03-31')
end_time = pd.to_datetime('2017-12-31')
start_year = start_time.year
end_year = end_time.year

# --- Domain file ("mask" and "area" will be used) --- #
domain_nc = '/civil/hydro/ymao/data_assim/param/vic/ArkRed/ArkRed.domain.nc'

# --- RVIC output param remap nc files --- #
# "{}" will be replaced by site name
rvic_subbasin_nc = '/civil/hydro/ymao/data_assim/param/RVIC/ArkRed/parameter/' \
                   'param_run_output.8sites_Crow2017/temp/remapped/remapUH_{}.nc'
    
# --- VIC forcing file basedir ("YYYY.nc" will be appended) --- #
force_basedir = '/civil/hydro/ymao/data_assim/forcing/vic/NLDAS-2/ArkRed/force.'

# --- VIC opnloop output history nc (this is to calculate baseflow fraction) --- # 
vic_openloop_hist_nc = \
    '/civil/hydro/ymao/data_assim/output/vic/ArkRed/LAI_from_veglib/openloop.NLDAS2.2015_2017/' \
    'history/history.openloop.2015-03-31-00000.nc'

# --- Output --- #
output_dir = '/civil/hydro/ymao/data_assim/tools/process_evaluation_data_ArkRed/' \
             'output/20180424.ArkRed.smap.LAI_from_veglib.qc_no_winter/' \
             'sm_0.5.prec_0.3.N32.EnKF_perturbed_force'

In [15]:
# ===================================================== #
# Load USGS data
# ===================================================== #
# --- Load site info --- #
df_site_info = pd.read_csv(site_info_csv, dtype={'site_no': str})
dict_sites = {}  # {site: site_no}
for i in df_site_info.index:
    site = df_site_info.loc[i, 'short_name']
    site_no = df_site_info.loc[i, 'site_no']
    dict_sites[site] = site_no
    
# --- Load USGS streamflow data --- #
dict_flow_usgs = {}  # {site: ts}
for site in dict_sites.keys():
    print('Loading USGS data for site {}...'.format(site))
    site_no = dict_sites[site]
    filename = os.path.join(usgs_data_dir, '{}.txt'.format(site_no))
    ts_flow = read_USGS_data(filename, columns=[1], names=['flow'])['flow']
    dict_flow_usgs[site] = ts_flow.truncate(before=start_time, after=end_time)

# --- Get USGS drainage area (mi2) --- #
dict_usgs_drainage_area = {}  # {site: area}
for i in df_site_info.index:
    site = df_site_info.loc[i, 'short_name']
    drainage_area = df_site_info.loc[i, 'drain_area_va']
    dict_usgs_drainage_area[site] = drainage_area * 1.60934 * 1.60934  # convert [mi2] to [km2]

Loading USGS data for site illinois...
Loading USGS data for site walnut...
Loading USGS data for site deep...
Loading USGS data for site bird...
Loading USGS data for site chikaskia...
Loading USGS data for site spring...
Loading USGS data for site arkansas...
Loading USGS data for site ninnescah...


In [32]:
# ===================================================== #
# Load synthetic results
# ===================================================== #
# --- Load PER --- #
# Surface runoff
da_synth_openloop_rmseLog_runoff = xr.open_dataset(
    os.path.join(synth_openloop_data_dir, 'rmseLog_openloop_dailyRunoff.nc'))['rmse']
da_synth_rmseLog_runoff = xr.open_dataset(
    os.path.join(synth_data_dir, 'rmseLog_EnKF_runoffDaily.nc'))['rmse']
da_per_synth_runoff = (1 - da_synth_rmseLog_runoff / da_synth_openloop_rmseLog_runoff) * 100
# Baseflow
da_synth_openloop_rmseLog_baseflow = xr.open_dataset(
    os.path.join(synth_openloop_data_dir, 'rmseLog_openloop_dailyBaseflow.nc'))['rmse']
da_synth_rmseLog_baseflow = xr.open_dataset(
    os.path.join(synth_data_dir, 'rmseLog_EnKF_baseflowDaily.nc'))['rmse']
da_per_synth_baseflow = (1 - da_synth_rmseLog_baseflow / da_synth_openloop_rmseLog_baseflow) * 100
# Total runoff
da_synth_openloop_rmseLog_totrunoff = xr.open_dataset(
    os.path.join(synth_openloop_data_dir, 'rmseLog_openloop_dailyTotrunoff.nc'))['rmse']
da_synth_rmseLog_totrunoff = xr.open_dataset(
    os.path.join(synth_data_dir, 'rmseLog_EnKF_totrunoffDaily.nc'))['rmse']
da_per_synth_totrunoff = (1 - da_synth_rmseLog_totrunoff / da_synth_openloop_rmseLog_totrunoff) * 100

# --- Load PSR --- #
# Surface runoff
da_synth_crps_runoff_zero_update = xr.open_dataset(
    os.path.join(synth_zero_updata_data_dir, 'crps_EnKF_runoff_daily_log.nc'))['crps']
da_synth_crps_runoff = xr.open_dataset(
    os.path.join(synth_data_dir, 'crps_EnKF_runoff_daily_log.nc'))['crps']
da_psr_synth_runoff = (1 - da_synth_crps_runoff / da_synth_crps_runoff_zero_update) * 100
# Baseflow
da_synth_crps_baseflow_zero_update = xr.open_dataset(
    os.path.join(synth_zero_updata_data_dir, 'crps_EnKF_baseflow_daily_log.nc'))['crps']
da_synth_crps_baseflow = xr.open_dataset(
    os.path.join(synth_data_dir, 'crps_EnKF_baseflow_daily_log.nc'))['crps']
da_psr_synth_baseflow = (1 - da_synth_crps_baseflow / da_synth_crps_baseflow_zero_update) * 100
# Total runoff
da_synth_crps_totrunoff_zero_update = xr.open_dataset(
    os.path.join(synth_zero_updata_data_dir, 'crps_EnKF_totrunoff_daily_log.nc'))['crps']
da_synth_crps_totrunoff = xr.open_dataset(
    os.path.join(synth_data_dir, 'crps_EnKF_totrunoff_daily_log.nc'))['crps']
da_psr_synth_totrunoff = (1 - da_synth_crps_totrunoff / da_synth_crps_totrunoff_zero_update) * 100

In [17]:
# ===================================================== #
# Load and process routed data
# ===================================================== #
# --- Load openloop data --- #
df_openloop, dict_outlet = read_RVIC_output(openloop_nc)

# --- Load ensemble data --- #
list_da_ensemble = []
for i in range(N):
    filename = ensemble_basenc.format(i+1)
    df, dict_outlet = read_RVIC_output(filename)
    da = xr.DataArray(df, dims=['time', 'site'])
    list_da_ensemble.append(da)
# Concat all ensemble members
da_ensemble = xr.concat(list_da_ensemble, dim='N')

# --- Load zero-update data --- #
list_da_ensemble = []
for i in range(N):
    filename = zero_update_ensemble_basenc.format(i+1)
    df, dict_outlet = read_RVIC_output(filename)
    da = xr.DataArray(df, dims=['time', 'site'])
    list_da_ensemble.append(da)
# Concat all ensemble members
da_zero_update_ensemble = xr.concat(list_da_ensemble, dim='N')

# --- Shift all routed data data to local time --- #
df_openloop.index = df_openloop.index - pd.DateOffset(hours=time_lag)
da_ensemble['time'] = pd.to_datetime(da_ensemble['time'].values) - pd.DateOffset(hours=time_lag)
da_zero_update_ensemble['time'] = \
    pd.to_datetime(da_zero_update_ensemble['time'].values) - pd.DateOffset(hours=time_lag)

# --- Average all routed data to daily (of local time) --- #
df_openloop_daily = df_openloop.resample('1D', how='mean')
da_ensemble_daily = da_ensemble.resample('1D', dim='time', how='mean')
da_zero_update_ensemble_daily = da_zero_update_ensemble.resample('1D', dim='time', how='mean')

# --- Calculate ensemble mean --- #
da_ensMean_daily = da_ensemble_daily.mean(dim='N')
da_zero_update_ensMean_daily = da_zero_update_ensemble_daily.mean(dim='N')

/civil/hydro/ymao/anaconda3/envs/da2/lib/python3.5/site-packages/ipykernel_launcher.py:33: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
/civil/hydro/ymao/anaconda3/envs/da2/lib/python3.5/site-packages/ipykernel_launcher.py:34: DeprecationWarning: 
.resample() has been modified to defer calculations. Instead of passing 'dim' and 'how="mean", instead consider using .resample(time="1D").mean() 
/civil/hydro/ymao/anaconda3/envs/da2/lib/python3.5/site-packages/xarray/core/common.py:619: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  label=label, base=base)
/civil/hydro/ymao/anaconda3/envs/da2/lib/python3.5/site-packages/ipykernel_launcher.py:35: DeprecationWarning: 
.resample() has been modified to defer calculations. Instead of passing 'dim' and 'how="mean", instead consider using .resample(time="1D").mean() 


In [18]:
# ===================================================== #
# Load basin information
# ===================================================== #
ds_domain = xr.open_dataset(domain_nc)
da_area = ds_domain['area']
da_domain = ds_domain['mask']

# --- Basin domain --- #
dict_da_frac = {}
for site in dict_sites.keys():
    da_frac = xr.open_dataset(rvic_subbasin_nc.format(site))['fraction']
    dict_da_frac[site] = da_frac

# --- Basin area --- #
dict_basin_area = {}
for site in dict_sites.keys():
    basin_area = float(da_area.where(dict_da_frac[site]>0).sum())  # [m2]
    basin_area = basin_area / 1000 / 1000  # convert to [km2]
    dict_basin_area[site] = basin_area
    print(site, basin_area)

illinois 3276.8724308692413
walnut 5043.581406236542
deep 4863.3081974647885
bird 3103.7095348407056
chikaskia 4310.18946663719
spring 5075.144257650728
arkansas 3489.61244685949
ninnescah 1983.1028074048015


/civil/hydro/ymao/anaconda3/envs/da2/lib/python3.5/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):


In [20]:
# ===================================================== #
# Calculate basin baseflow fraction (average baseflow / total runoff)
# ===================================================== #
# --- Load openloop VIC runoff --- #
ds_openloop = xr.open_dataset(vic_openloop_hist_nc)
da_totrunoff = ds_openloop['OUT_RUNOFF'] + ds_openloop['OUT_BASEFLOW']
da_baseflow = ds_openloop['OUT_BASEFLOW']
# --- Calculate basin-avg baseflow fraction --- #
dict_baseflow_fraction = {}
for site in dict_sites.keys():
    # Baseflow sum
    da_basin_baseflow = da_baseflow.where(dict_da_frac[site]>0)
    basin_baseflow = (da_basin_baseflow * da_area).sum(
        dim='lat').sum(dim='lon').mean(dim='time').values  # [mm*m2/step]
    # Total runoff sum
    da_basin_totrunoff = da_totrunoff.where(dict_da_frac[site]>0)
    basin_totrunoff = (da_basin_totrunoff * da_area).sum(
        dim='lat').sum(dim='lon').mean(dim='time').values  # [mm*m2/step]
    # Baseflow fraction
    baseflow_fraction = basin_baseflow / basin_totrunoff
    dict_baseflow_fraction[site] = baseflow_fraction

/civil/hydro/ymao/anaconda3/envs/da2/lib/python3.5/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):


In [34]:
# ===================================================== #
# Calculate and save subbasin metrics for both synthetic and real-data cases
# ===================================================== #
for site in dict_sites.keys():
    # --- Some calculation --- #
    ts_usgs = dict_flow_usgs[site]
    ts_openloop = df_openloop_daily.loc[:, site]
    ts_usgs = ts_usgs[ts_openloop.index].dropna()
    ts_openloop = ts_openloop[ts_usgs.index].dropna()
    ts_ensMean = da_ensMean_daily.sel(site=site).to_series()[ts_usgs.index].dropna()
    ensemble_daily = da_ensemble_daily.sel(
        site=site, time=ts_usgs.index).transpose('time', 'N').values
    zero_update_ensemble_daily = da_zero_update_ensemble_daily.sel(
        site=site, time=ts_usgs.index).transpose('time', 'N').values
    
    # --- Real-data routed streamflow --- #
    # PER
    rmseLog_openloop = rmse(np.log(ts_openloop+1), np.log(ts_usgs+1))
    rmseLog_ensMean = rmse(np.log(ts_ensMean+1), np.log(ts_usgs+1))
    per = (1 - rmseLog_ensMean / rmseLog_openloop) * 100
    # PSR
    crps_ens = crps(ts_usgs, ensemble_daily)
    crps_zero_update = crps(ts_usgs, zero_update_ensemble_daily)
    psr = (1 - crps_ens / crps_zero_update) * 100
    # Normalized ensemble skill (NENSK)
    nensk_ens = nensk(ts_usgs, ensemble_daily)

    # --- Synthetic, subbasin median --- #
    # PER
    per_synth_totrunoff = da_per_synth_totrunoff.where(dict_da_frac[site]>0).median().values
    per_synth_surfacerunoff = da_per_synth_runoff.where(dict_da_frac[site]>0).median().values
    per_synth_baseflow = da_per_synth_baseflow.where(dict_da_frac[site]>0).median().values
    # PSR
    psr_synth_totrunoff = da_psr_synth_totrunoff.where(dict_da_frac[site]>0).median().values
    psr_synth_surfacerunoff = da_psr_synth_runoff.where(dict_da_frac[site]>0).median().values
    psr_synth_baseflow = da_psr_synth_baseflow.where(dict_da_frac[site]>0).median().values
    
    # --- Print and save results --- #
    print('{} \treal-data \tsynth \tsynth_surfaceRunoff \tsynth_baseflow'.format(
        site))
    print('PER: \t\t{:.1f}% \t\t{:.1f}% \t\t{:.1f}% \t\t{:.1f}%'.format(
        per, per_synth_totrunoff, per_synth_surfacerunoff, per_synth_baseflow))
    print('PSR: \t\t{:.1f}% \t\t{:.1f}% \t\t{:.1f}% \t\t{:.1f}%'.format(
        psr, psr_synth_totrunoff, psr_synth_surfacerunoff, psr_synth_baseflow))
    print('Baseflow fraction = {:.3f}'.format(dict_baseflow_fraction[site]))
    print('\n')

/civil/hydro/ymao/anaconda3/envs/da2/lib/python3.5/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):


illinois 	real-data 	synth 	synth_surfaceRunoff 	synth_baseflow
PER: 		3.0% 		16.4% 		0.1% 		19.2%
PSR: 		0.1% 		7.1% 		1.0% 		9.2%
Baseflow fraction = 0.626




/civil/hydro/ymao/anaconda3/envs/da2/lib/python3.5/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):


walnut 	real-data 	synth 	synth_surfaceRunoff 	synth_baseflow
PER: 		7.0% 		15.9% 		2.5% 		20.1%
PSR: 		-0.6% 		13.0% 		2.0% 		20.2%
Baseflow fraction = 0.451




/civil/hydro/ymao/anaconda3/envs/da2/lib/python3.5/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):


...

/civil/hydro/ymao/anaconda3/envs/da2/lib/python3.5/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):


spring 	real-data 	synth 	synth_surfaceRunoff 	synth_baseflow
PER: 		5.8% 		17.3% 		1.3% 		19.4%
PSR: 		2.2% 		11.6% 		1.7% 		15.0%
Baseflow fraction = 0.639




/civil/hydro/ymao/anaconda3/envs/da2/lib/python3.5/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):


arkansas 	real-data 	synth 	synth_surfaceRunoff 	synth_baseflow
PER: 		3.9% 		50.7% 		3.2% 		71.7%
PSR: 		-0.4% 		8.4% 		2.9% 		24.6%
Baseflow fraction = 0.282


ninnescah 	real-data 	synth 	synth_surfaceRunoff 	synth_baseflow
PER: 		-2.0% 		8.6% 		3.7% 		39.5%
PSR: 		0.1% 		8.3% 		4.6% 		18.8%
Baseflow fraction = 0.320




/civil/hydro/ymao/anaconda3/envs/da2/lib/python3.5/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):


## Functions

In [5]:
def read_RVIC_output(filepath, output_format='array', outlet_ind=-1):
    ''' This function reads RVIC output netCDF file

    Input:
        filepath: path of the output netCDF file
        output_format: 'array' or 'grid' (currently only support 'array')
        outlet_ind: index of the outlet to be read (index starts from 0); -1 for reading all outlets

    Return:
        df - a DataFrame containing streamflow [unit: cfs]; column name(s): outlet name
        dict_outlet - a dictionary with outlet name as keys; [lat lon] as content

    '''
    
    ds = xr.open_dataset(filepath)

    #=== Read in outlet names ===#
    outlet_names = [outlet_name.decode('utf-8')
                    for outlet_name in ds['outlet_name'].values]

    #=== Read in outlet lat lon ===#
    dict_outlet = {}
    # If read all outlets
    if outlet_ind==-1:
        for i, name in enumerate(outlet_names):
            dict_outlet[name] = [ds['lat'].values[i], ds['lon'].values[i]]
    # If read one outlet
    else:
        dict_outlet[outlet_names[outlet_ind]] = \
                        [ds['lat'].values[outlet_ind], ds['lon'].values[outlet_ind]]

    #=== Read in streamflow variable ===#
    flow = ds['streamflow'].values
    flow = flow * np.power(1000/25.4/12, 3)  # convert m3/s to cfs
    # If read all outlets
    if outlet_ind==-1:
        df = pd.DataFrame(flow, index=ds.coords['time'].values, columns=outlet_names)
    # If read one outlet
    else:
        df = pd.DataFrame(flow[:,outlet_ind], index=ds.coords['time'].values, \
                          columns=[outlet_names[outlet_ind]])

    return df, dict_outlet

In [6]:
def read_USGS_data(file, columns, names):
    '''This function reads USGS streamflow from the directly downloaded format (date are in the 3rd columns)

    Input:
        file: directly downloaded streamflow file path [str]
        columns: a list of data colomn numbers, starting from 1.
            E.g., if the USGS original data has three variables: max_flow, min_flow,
            mean_flow, and the desired variable is mean_flow, then columns = [3]
        names: a list of data column names. E.g., ['mean_flow']; must the same length as columns

    Return:
        a pd.DataFrame object with time as index and data columns (NaN for missing data points)

    Note: returned data and flow might not be continuous if there is missing data!!!

    '''
    ndata = len(columns)
    if ndata != len(names):  # check input validity
        raise ValueError("Input arguments 'columns' and 'names' must have same length!")

    f = open(file, 'r')
    date_array = []
    data = []
    for i in range(ndata):
        data.append([])
    while 1:
        line = f.readline().rstrip("\n")  # read in one line
        if line=="":
                break
        line_split = line.split('\t')
        if line_split[0]=='USGS':  # if data line
                date_string = line_split[2]  # read in date string
                date = dt.datetime.strptime(date_string, "%Y-%m-%d")  # convert date to dt object
                date_array.append(date)

                for i in range(ndata):  # for each desired data variable
                        col = columns[i]
                        if line_split[3+(col-1)*2] == '':  # if data is missing
                                value = np.nan
                        elif line_split[3+(col-1)*2] == 'Ice':  # if data is 'Ice'
                                value = np.nan
                        else:  # if data is not missing
                                value = float(line_split[3+(col-1)*2])
                        data[i].append(value)

    data = np.asarray(data).transpose()
    df = pd.DataFrame(data, index=date_array, columns=names)
    return df

In [7]:
def kge(sim, obs):
    ''' Calculate Kling-Gupta Efficiency (function from Oriana) '''

    std_sim = np.std(sim)
    std_obs = np.std(obs)
    mean_sim = sim.mean(axis=0)
    mean_obs = obs.mean(axis=0)
    r_array = np.corrcoef(sim.values, obs.values)
    r = r_array[0,1]
    relvar = std_sim/std_obs
    bias = mean_sim/mean_obs
    kge = 1-np.sqrt(np.square(r-1) + np.square(relvar-1) + np.square(bias-1))
    return kge

In [8]:
def nse(sim, obs):
    ''' Calcualte Nash–Sutcliffe efficiency'''
    
    obs_mean = np.mean(obs)
    nse = 1 - np.sum(np.square(sim - obs)) / np.sum(np.square(obs - obs_mean))
    return nse

In [9]:
def rmse(true, est):
    ''' Calculates RMSE of an estimated variable compared to the truth variable

    Parameters
    ----------
    true: <np.array>
        A 1-D array of time series of true values
    est: <np.array>
        A 1-D array of time series of estimated values (must be the same length of true)

    Returns
    ----------
    rmse: <float>
        Root mean square error

    Require
    ----------
    numpy
    '''

    rmse = np.sqrt(sum((est - true)**2) / len(true))
    return rmse

In [10]:
def crps(truth, ensemble):
    ''' Calculate mean CRPS of an ensemble time series
    Parameters
    ----------
    truth: <np.array>
        A 1-D array of truth time series
        Dimension: [n]
    ensemble: <np.array>
        A 2-D array of ensemble time series
        Dimension: [n, N], where N is ensemble size; n is time series length
        
    Returns
    ----------
    crps: <float>
        Time-series-mean CRPS
        
    Require
    ----------
    import properscoring as ps
    '''
    
    array_crps = np.asarray([ps.crps_ensemble(truth[t], ensemble[t, :]) for t in range(len(truth))])
    crps = array_crps.mean()
    
    return crps

In [11]:
def bias_ensemble_norm_var(truth, ensemble):
    ''' Calculate variance of normalized bias of an ensemble time series.
    Specifically, at each time step t, mean bias is normalized by ensemble spread:
            bias_norm(t) = mean_bias / std(ensemble)
    Then average over all time steps:
            bias_norm = mean(bias_norm(t))
            
    Parameters
    ----------
    truth: <np.array>
        A 1-D array of truth time series
        Dimension: [n]
    ensemble: <np.array>
        A 2-D array of ensemble time series
        Dimension: [n, N], where N is ensemble size; n is time series length
        
    Returns
    ----------
    bias_ensemble_norm_var: <float>
        Time-series-mean ensemble-normalized bias
        
    Require
    ----------
    import properscoring as ps
    '''
    
    mean_bias = ensemble.mean(axis=1) - truth  # [n]
    std_ensemble = ensemble.std(axis=1)  # [n]
    bias_ensemble_norm_var = (mean_bias / std_ensemble).var()
    
    return bias_ensemble_norm_var

In [12]:
def nensk(truth, ensemble):
    ''' Calculate the ratio of temporal-mean ensemble skill to temporal-mean ensemble spread:
            nensk = <ensk> / <ensp>
    where <ensk> is temporal average of: ensk(t) = (ensmean - truth)^2
          <ensp> is temperal average of: ensp(t) = mean((ens_i - ensmean)^2) = var(ens_i)

    Parameters
    ----------
    truth: <np.array>
        A 1-D array of truth time series
        Dimension: [n]
    ensemble: <np.array>
        A 2-D array of ensemble time series
        Dimension: [n, N], where N is ensemble size; n is time series length

    Returns
    ----------
    nensk: <float>
        Normalized ensemble skill
    '''

    ensk = np.square((ensemble.mean(axis=1) - truth))  # [n]
    ensp = ensemble.var(axis=1)  # [n]
    nensk = np.mean(ensk) / np.mean(ensp)

    return nensk